In [ ]:
import numpy as np
import random
import torch
import torch.nn as nn
import torch.optim as optim

# Custom CartPole Environment
class CustomCartPole:
    def __init__(self):
        self.gravity = 9.8
        self.mass_cart = 1.0
        self.mass_pole = 0.1
        self.total_mass = self.mass_cart + self.mass_pole
        self.length = 0.5
        self.pole_mass_length = self.mass_pole * self.length
        self.force_mag = 10.0
        self.tau = 0.02
        self.theta_threshold_radians = 12 * 2 * np.pi / 360
        self.x_threshold = 2.4
        self.state = None
        self.steps_beyond_done = None

    def reset(self):
        self.state = np.random.uniform(low=-0.05, high=0.05, size=(4,))
        self.steps_beyond_done = None
        return np.array(self.state, dtype=np.float32)

    def step(self, action):
        x, x_dot, theta, theta_dot = self.state
        force = self.force_mag if action == 1 else -self.force_mag
        costheta = np.cos(theta)
        sintheta = np.sin(theta)
        temp = (force + self.pole_mass_length * theta_dot**2 * sintheta) / self.total_mass
        theta_acc = (self.gravity * sintheta - costheta * temp) / (self.length * (4.0/3.0 - self.mass_pole * costheta**2 / self.total_mass))
        x_acc = temp - self.pole_mass_length * theta_acc * costheta / self.total_mass
        x = x + self.tau * x_dot
        x_dot = x_dot + self.tau * x_acc
        theta = theta + self.tau * theta_dot
        theta_dot = theta_dot + self.tau * theta_acc
        self.state = (x, x_dot, theta, theta_dot)
        done = bool(x < -self.x_threshold or x > self.x_threshold or theta < -self.theta_threshold_radians or theta > self.theta_threshold_radians)
        reward = 1.0 if not done else 0.0
        return np.array(self.state, dtype=np.float32), reward, done, {}

# Q-Network for DQN
class QNetwork(nn.Module):
    def __init__(self, state_dim, action_dim):
        super(QNetwork, self).__init__()
        self.fc1 = nn.Linear(state_dim, 128)
        self.fc2 = nn.Linear(128, 128)
        self.fc3 = nn.Linear(128, action_dim)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        return self.fc3(x)

# DQN Agent
class DQNAgent:
    def __init__(self, state_dim, action_dim, lr=1e-3, gamma=0.99, epsilon_decay=0.995, batch_size=64, buffer_capacity=10000):
        self.state_dim = state_dim
        self.action_dim = action_dim
        self.q_network = QNetwork(state_dim, action_dim)
        self.target_network = QNetwork(state_dim, action_dim)
        self.optimizer = optim.Adam(self.q_network.parameters(), lr=lr)
        self.replay_buffer = []
        self.buffer_capacity = buffer_capacity
        self.batch_size = batch_size
        self.gamma = gamma
        self.epsilon = 1.0
        self.epsilon_decay = epsilon_decay
        self.epsilon_min = 0.01

    def act(self, state):
        if random.random() < self.epsilon:
            return random.randint(0, self.action_dim - 1)
        state_tensor = torch.FloatTensor(state).unsqueeze(0)
        with torch.no_grad():
            return torch.argmax(self.q_network(state_tensor)).item()

    def store_transition(self, transition):
        if len(self.replay_buffer) >= self.buffer_capacity:
            self.replay_buffer.pop(0)
        self.replay_buffer.append(transition)

    def sample_batch(self):
        indices = np.random.choice(len(self.replay_buffer), self.batch_size)
        batch = [self.replay_buffer[idx] for idx in indices]
        return batch

    def learn(self):
        if len(self.replay_buffer) < self.batch_size:
            return
        batch = self.sample_batch()
        states, actions, rewards, next_states, dones = zip(*batch)
        states = torch.FloatTensor(states)
        actions = torch.LongTensor(actions)
        rewards = torch.FloatTensor(rewards)
        next_states = torch.FloatTensor(next_states)
        dones = torch.FloatTensor(dones)
        q_values = self.q_network(states).gather(1, actions.unsqueeze(1)).squeeze()
        with torch.no_grad():
            max_next_q_values = self.target_network(next_states).max(1)[0]
            target_q_values = rewards + self.gamma * max_next_q_values * (1 - dones)
        loss = nn.MSELoss()(q_values, target_q_values)
        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()

    def update_target_network(self):
        self.target_network.load_state_dict(self.q_network.state_dict())

# Initialize environment and agent
env = CustomCartPole()
state_dim = 4
action_dim = 2
agent = DQNAgent(state_dim, action_dim)
num_episodes = 500

# Training loop
for episode in range(num_episodes):
    state = env.reset()
    total_reward = 0
    reason_for_termination = None
    for t in range(500):
        action = agent.act(state)
        next_state, reward, done, _ = env.step(action)
        cart_position = next_state[0]
        pole_angle = next_state[2]
        if abs(cart_position) > 2.4:
            reason_for_termination = f"Cart moved out of bounds: {cart_position:.2f}m"
        elif abs(pole_angle) > 0.209:
            reason_for_termination = f"Pole angle exceeded: {pole_angle:.2f} radians"
        agent.store_transition((state, action, reward, next_state, done))
        state = next_state
        total_reward += reward
        agent.learn()
        if done:
            break
    agent.update_target_network()
    agent.epsilon = max(agent.epsilon_min, agent.epsilon * agent.epsilon_decay)
    if reason_for_termination is None:
        print(f"Episode {episode}, Total Reward: {total_reward}, Episode completed successfully")
    else:
        print(f"Episode {episode}, Total Reward: {total_reward}, Terminated due to: {reason_for_termination}")
